# FloPy - Voronoi grid model for VTK export

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mticker
from matplotlib import colors
import shapely
from shapely.geometry import Polygon, LineString
import flopy
from flopy.discretization import StructuredGrid, VertexGrid
from flopy.utils.triangle import Triangle
from flopy.utils.voronoi import VoronoiGrid
from flopy.utils.gridgen import Gridgen
import flopy.plot.styles as styles

In [ ]:
# import all plot style information from defaults.py
from defaults import *

In [ ]:
# import the fine topography
fine_topo = flopy.utils.Raster.load("./grid_data/fine_topo.asc")

In [ ]:
fine_topo.plot();

In [ ]:
Lx = 180000
Ly = 100000
nlay = 5
dv0 = 5.0
extent = (0, Lx, 0, Ly)
vmin, vmax = 0.0, 100.0

In [ ]:
temp_path = "./temp"
if not os.path.isdir(temp_path):
    os.mkdir(temp_path)

# Basin Example

In [ ]:
boundary_polygon = string2geom(boundary)
print("len boundary", len(boundary_polygon))
bp = np.array(boundary_polygon)

sgs = [
    string2geom(sg) for sg in (streamseg1, streamseg2, streamseg3, streamseg4)
]

river_colors = ("blue", "cyan", "green", "magenta")
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot()
ax.set_aspect("equal")

ax.plot(bp[:, 0], bp[:, 1], "ko-")
for idx, sg in enumerate(sgs):
    print("Len segment: ", len(sg))
    sa = np.array(sg)
    ax.plot(sa[:, 0], sa[:, 1], ls="-", color=river_colors[idx], marker="o")

### Create voronoi grid

In [ ]:
maximum_area = 5000 * 5000

nodes = []
for sg in sgs:
    sg_densify = densify_geometry(sg, 2000)
    nodes += sg_densify
nodes = np.array(nodes)

tri = Triangle(
    maximum_area=maximum_area, angle=30, nodes=nodes, model_ws=temp_path
)
poly = bp
tri.add_polygon(poly)
tri.build(verbose=False)

# create vor object and VertexGrid
vor = VoronoiGrid(tri)
gridprops = vor.get_gridprops_vertexgrid()
idomain = np.ones((nlay, vor.ncpl), dtype=int)
voronoi_grid = VertexGrid(**gridprops, nlay=nlay, idomain=idomain)

In [ ]:
areas = []
for idx in range(voronoi_grid.ncpl):
    vertices = np.array(voronoi_grid.get_cell_vertices(idx))
    area = Polygon(vertices).area
    areas.append(area)
areas = np.array(areas)
areas.min(), areas.max()

In [ ]:
top_vg = fine_topo.resample_to_grid(
    voronoi_grid,
    band=fine_topo.bands[0],
    method="linear",
    extrapolate_edges=True,
)

In [ ]:
ixs = flopy.utils.GridIntersect(voronoi_grid, method="vertex")
cellids = []
for sg in sgs:
    v = ixs.intersect(LineString(sg), sort_by_cellid=True)
    cellids += v["cellids"].tolist()
intersection_vg = np.zeros(voronoi_grid.shape[1:])
for loc in cellids:
    intersection_vg[loc] = 1

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
pmv = flopy.plot.PlotMapView(modelgrid=voronoi_grid)
ax.set_aspect("equal")
pmv.plot_array(top_vg)
pmv.plot_array(
    intersection_vg,
    masked_values=[
        0,
    ],
    alpha=0.2,
    cmap="Reds_r",
)
# pmv.plot_grid()
pmv.plot_inactive()
ax.plot(bp[:, 0], bp[:, 1], "k-")
for sg in sgs:
    sa = np.array(sg)
    ax.plot(sa[:, 0], sa[:, 1], "b-")

cg = pmv.contour_array(top_vg, levels=levels, linewidths=0.3, colors="0.75")

### build a model

In [ ]:
ixs = flopy.utils.GridIntersect(voronoi_grid, method="vertex")

In [ ]:
drn_intersection = []
drn_cellids = []
drn_lengths = []
for sg in sgs:
    v = ixs.intersect(LineString(sg), sort_by_cellid=True)
    drn_intersection.append(v)
    drn_cellids += v["cellids"].tolist()
    drn_lengths += v["lengths"].tolist()

In [ ]:
leakance = 1.0 / (0.5 * dv0)  # kv / b
drn_data = []
for node, length in zip(drn_cellids, drn_lengths):
    x = voronoi_grid.xcellcenters[node]
    width = 5.0 + (14.0 / Lx) * (Lx - x)
    conductance = leakance * length * width
    drn_data.append((0, node, top_vg[node], conductance))
drn_data[:10]

In [ ]:
# groundwater discharge to surface
gw_discharge_data = []
for node in range(voronoi_grid.ncpl):
    if node not in drn_cellids:
        vertices = np.array(voronoi_grid.get_cell_vertices(node))
        conductance = leakance * Polygon(vertices).area
        gw_discharge_data.append(
            (0, node, top_vg[node] - 0.5, conductance, 1.0)
        )
gw_discharge_data[:10]

In [ ]:
topc = np.zeros((nlay, vor.ncpl), dtype=float)
botm = np.zeros((nlay, vor.ncpl), dtype=float)
dv = dv0
topc[0] = top_vg.copy()
botm[0] = topc[0] - dv
for idx in range(1, nlay):
    dv *= 1.5
    topc[idx] = botm[idx - 1]
    botm[idx] = topc[idx] - dv

In [ ]:
for k in range(nlay):
    print((topc[k] - botm[k]).mean())

In [ ]:
exe_name = "/Users/jdhughes/Documents/Development/modflow6/modflow6/bin/mf6"
sim = flopy.mf6.MFSimulation(
    sim_name="create_vtk",
    sim_ws="temp_vtk",
    exe_name="mf6",
)

tdis = flopy.mf6.ModflowTdis(sim)
ims = flopy.mf6.ModflowIms(
    sim, linear_acceleration="bicgstab", complexity="simple"
)
gwf = flopy.mf6.ModflowGwf(
    sim, save_flows=True, newtonoptions="NEWTON UNDER_RELAXATION"
)

dis = flopy.mf6.ModflowGwfdisv(
    gwf,
    nlay=nlay,
    ncpl=vor.ncpl,
    icelltype=1,
    nvert=vor.get_disv_gridprops()["nvert"],
    vertices=vor.get_disv_gridprops()["vertices"],
    cell2d=vor.get_disv_gridprops()["cell2d"],
    top=top_vg,
    botm=botm,
)

ic = flopy.mf6.ModflowGwfic(gwf, strt=top_vg.max())
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    icelltype=1,
    k=1.0,
)
rch = flopy.mf6.ModflowGwfrcha(
    gwf,
    recharge=0.000001,
)
drn = flopy.mf6.ModflowGwfdrn(
    gwf,
    stress_period_data=drn_data,
    pname="river",
)
drn_gwd = flopy.mf6.ModflowGwfdrn(
    gwf,
    auxiliary=["depth"],
    auxdepthname="depth",
    stress_period_data=gw_discharge_data,
    pname="gwd",
)
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{gwf.name}.hds",
    budget_filerecord=f"{gwf.name}.cbc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

In [ ]:
sim.write_simulation()
sim.run_simulation()

In [ ]:
def plot_river(
    ax=None,
    lw=1.0,
):
    if ax is None:
        ax = plt.gca()
    for sg in sgs:
        sg_densify = np.array(densify_geometry(sg, 2000))
        ax.plot(sg_densify[:, 0], sg_densify[:, 1], **river_dict)
    return ax

In [ ]:
head = gwf.output.head().get_data().squeeze()

In [ ]:
head[0].min(), head[0].max()

In [ ]:
gwf.output.methods()

In [ ]:
cbc = gwf.output.budget()
cbc.list_unique_records(), cbc.list_unique_packages()

In [ ]:
spdis = cbc.get_data(text="DATA-SPDIS")[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)
qx.shape

In [ ]:
riv_q, gwd_q = cbc.get_data(text="DRN", full3D=True)
riv_q.shape, gwd_q.shape

In [ ]:
riv_q_loc = np.zeros(riv_q.shape, dtype=int)
gwd_q_loc = np.zeros(gwd_q.shape, dtype=int)

In [ ]:
drn_q_loc = np.zeros(gwd_q.shape, dtype=int)
drn_q_loc[riv_q < 0.0] = 1
drn_q_loc[gwd_q < 0.0] = 2

In [ ]:
idx = riv_q < 0.0
riv_q_loc[idx] = 1

In [ ]:
idx = gwd_q < 0.0
gwd_q_loc[idx] = 1

In [ ]:
dry_cell_loc = np.zeros(head.shape, dtype=int)
dry_cell_loc[head < botm] = 1

In [ ]:
upper_active_layer = np.ones(head.shape[1], dtype=int) * 99
for k in range(nlay):
    idx = (dry_cell_loc[k] < 1) & (upper_active_layer == 99)
    upper_active_layer[idx] = k

In [ ]:
qx_top = np.zeros(head.shape[1])
qy_top = np.zeros(head.shape[1])
for node, k in enumerate(upper_active_layer):
    qx_top[node] = qx[k, node]
    qy_top[node] = qy[k, node]

In [ ]:
fig = plt.figure(figsize=(10, 6), constrained_layout=True)
mm = flopy.plot.PlotMapView(model=gwf)
cb = mm.plot_array(head, ec="0.5")
plot_river(ax=mm.ax)
mm.plot_vector(qx_top, qy_top, normalize=True)
mm.ax.axhline(y=42500, lw=2, color="red")
mm.ax.axvline(x=72500, lw=2, color="red")
plt.colorbar(cb, orientation="horizontal");

In [ ]:
extent = mm.extent
extent

In [ ]:
fx = flopy.plot.PlotCrossSection(
    model=gwf, line={"line": [(0, 42500), (extent[1], 42500)]}
)
fx.plot_array(head, head=head)
fx.plot_grid()

In [ ]:
fx = flopy.plot.PlotCrossSection(
    model=gwf,
    line={"line": [(72500, extent[2]), (72500, extent[3])]},
)
fx.plot_array(head, head=head)
fx.plot_grid()

In [ ]:
topc.shape, head.shape

In [ ]:
dtw = topc[0] - head[0]
dtw.shape, dtw.min(), dtw.max()

In [ ]:
fig = plt.figure(figsize=(10, 6), constrained_layout=True)
mm = flopy.plot.PlotMapView(model=gwf)
cb = mm.plot_array(dtw)
cs = mm.contour_array(
    dtw,
    levels=[1, 5, 10],
    colors="white",
    linewidths=1,
)
mm.ax.clabel(cs, inline=1, fmt="%2.0f", fontsize=12, inline_spacing=0)
plot_river(ax=mm.ax)
plt.colorbar(cb, orientation="horizontal");

layer_cmap = colors.ListedColormap(["white", "green", "blue"])
drain_cmap = colors.ListedColormap(["red", "cyan"])
font_dict = {"fontsize": 5, "color": "black"}
contour_color = "black"

In [ ]:
fig = plt.figure(figsize=(10, 6), constrained_layout=True)
mm = flopy.plot.PlotMapView(model=gwf)
dp = mm.plot_array(
    drn_q_loc,
    masked_values=[0],
    cmap=drain_cmap,
    edgecolor="none",
    alpha=0.5,
    vmin=0.5,
    vmax=2.5,
)
al = mm.plot_array(
    upper_active_layer + 1,
    alpha=0.25,
    edgecolor="none",
    cmap=layer_cmap,
    vmin=0.5,
    vmax=3.5,
)
mm.plot_grid(color="black", lw=0.5)
plot_river(ax=mm.ax)

cax = mm.ax.inset_axes(
    [0.75, 0.85, 0.2, 0.05],
)
cbar = plt.colorbar(al, orientation="horizontal", cax=cax)
cbar.ax.tick_params(
    labelsize=5,
    labelcolor="black",
    color="white",
    length=6,
    pad=2,
)
cax.set_xticks([1, 2, 3])
cax.set_xticklabels([1, 2, 3])
cbar.ax.set_title(
    "Upper most active model layer",
    pad=2.5,
    loc="left",
    fontdict=font_dict,
)

cax = mm.ax.inset_axes(
    [0.02, 0.065, 0.2, 0.05],
)
cbar = plt.colorbar(dp, orientation="horizontal", cax=cax)
cbar.ax.tick_params(
    labelsize=5,
    labelcolor="black",
    color="white",
    length=6,
    pad=2,
)
cax.set_xticks([1, 2])
cax.set_xticklabels(["River", "Groundwater\nseepage"])
cbar.ax.set_title(
    "Discharge type",
    pad=2.5,
    loc="left",
    fontdict=font_dict,
);

In [ ]:
def set_map_axis_labels(ax, skip_ylabel=False):
    ax.set_xticks(np.arange(0, 200000, 50000))
    ax.set_xticklabels(np.arange(0, 200, 50))
    ax.set_xlabel("x position (km)")

    ax.set_yticks(np.arange(0, 150000, 50000))
    if skip_ylabel:
        ax.set_yticklabels([])
    else:
        ax.set_yticklabels(np.arange(0, 150, 50))
        ax.set_ylabel("y position (km)")

In [ ]:
def set_xsection_axis_labels(ax, skip_ylabel=False):
    xlim = ax.get_xlim()
    ax.set_ylim(-75, 125)
    ax.set_xticks(np.arange(0, xlim[1], 25000))
    ax.set_xticklabels(
        [f"{value:.0f}" for value in np.arange(0, xlim[1] / 1000, 25)]
    )
    ax.set_xlabel("cross-section distance (km)")

    ax.set_yticks(np.arange(-75, 150, 25))
    if skip_ylabel:
        ax.set_yticklabels([])
    else:
        ax.set_yticklabels(
            [f"{value:.0f}" for value in np.arange(-75, 150, 25)]
        )
        ax.set_ylabel("elevation (m)")

In [ ]:
figwidth = 17.15 / 2.54
figheight = 2.65 * (Ly / Lx) * 8.25 / 2.54
extent = (0, 180000, 0, 100000)

cbar_axis = [0.75, 0.825, 0.2, 0.05]

with styles.USGSMap():
    fig = plt.figure(figsize=(figwidth, figheight), constrained_layout=True)
    gs = gridspec.GridSpec(ncols=2, nrows=11, figure=fig)
    axs = [fig.add_subplot(gs[:5, 0])]
    axs.append(fig.add_subplot(gs[:5, 1]))
    axs.append(fig.add_subplot(gs[5:10, 0]))
    axs.append(fig.add_subplot(gs[5:10, 1]))
    axs.append(fig.add_subplot(gs[10:, :]))
    for ax in axs[:-1]:
        ax.set_axisbelow(False)

    # head
    ax = axs[0]
    ax.set_aspect("equal", "box")
    styles.heading(ax=ax, idx=0)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax)
    cb = mm.plot_array(head, ec="none", vmin=vmin, vmax=vmax)
    mm.plot_grid(**grid_dict)
    plot_river(ax=ax)
    cs = mm.contour_array(head, **contour_dict)
    ax.clabel(cs, **clabel_dict)

    q = mm.plot_vector(qx_top, qy_top, normalize=False)
    qk = plt.quiverkey(
        q,
        0.96,
        1.03,
        0.001,
        label="0.001 m/d",
        labelpos="W",
        labelcolor="black",
        fontproperties={"size": 8},
    )
    set_map_axis_labels(ax)

    # colorbar for head
    cax = mm.ax.inset_axes(
        cbar_axis,
    )
    cbar = plt.colorbar(cb, orientation="horizontal", cax=cax)
    cbar.ax.tick_params(
        labelsize=5,
        labelcolor="black",
        color="white",
        length=5,
        pad=2,
    )
    cbar.ax.set_title(
        "Head (m)",
        pad=2.5,
        loc="left",
        fontdict=font_dict,
    )

    # cross-section lines
    mm.ax.axhline(y=42500, lw=1, ls=":", color="red")
    styles.add_text(
        ax=ax,
        text="A",
        x=400,
        y=42600,
        transform=False,
        bold=True,
        color="red",
    )
    styles.add_text(
        ax=ax,
        text="A'",
        x=extent[1] - 400,
        y=42600,
        transform=False,
        bold=True,
        color="red",
    )
    mm.ax.plot([72500, 72500], [9000, 95000], lw=1, ls=":", color="red")
    styles.add_annotation(
        ax=ax,
        text="B",
        xy=(72500, 95000),
        xytext=(-15, -15),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="B'",
        xy=(72500, 9000),
        xytext=(15, 10),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )

    # cell-by-cell
    ax = axs[1]
    ax.set_aspect("equal", "box")
    styles.heading(ax=ax, idx=1)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax)
    aml = mm.plot_array(
        upper_active_layer + 1,
        edgecolor="none",
        cmap=layer_cmap,
        vmin=0.5,
        vmax=3.5,
    )
    dp = mm.plot_array(
        drn_q_loc,
        masked_values=[0],
        cmap=drain_cmap,
        edgecolor="none",
        vmin=0.5,
        vmax=2.5,
    )
    mm.plot_grid(**grid_dict)
    plot_river(ax=ax)
    set_map_axis_labels(ax, skip_ylabel=True)

    # color bar for B (model layer)
    cax = mm.ax.inset_axes(
        cbar_axis,
    )
    cbar = plt.colorbar(aml, orientation="horizontal", cax=cax)
    cbar.ax.tick_params(
        labelsize=5,
        labelcolor="black",
        color="none",
        length=5,
        pad=2,
    )
    cax.set_xticks([1, 2, 3])
    cax.set_xticklabels([1, 2, 3])
    cbar.ax.set_title(
        "Water-table layer",
        pad=2.5,
        loc="left",
        fontdict=font_dict,
    )

    # color bar for B (drain locations)
    cax = mm.ax.inset_axes(
        [0.02, 0.065, 0.2, 0.05],
    )
    cbar = plt.colorbar(dp, orientation="horizontal", cax=cax)
    cbar.ax.tick_params(
        labelsize=5,
        labelcolor="black",
        color="none",
        length=6,
        pad=2,
    )
    cax.set_xticks([1, 2])
    cax.set_xticklabels(["River", "Seepage"])
    cbar.ax.set_title(
        "Discharge type",
        pad=2.5,
        loc="left",
        fontdict=font_dict,
    )

    # east-west cross-section
    ax = axs[2]
    styles.heading(ax=ax, idx=2)
    fx = flopy.plot.PlotCrossSection(
        model=gwf, ax=ax, line={"line": [(0, 42500), (extent[1], 42500)]}
    )
    cb = fx.plot_array(head, head=head, vmin=vmin, vmax=vmax)
    fx.plot_grid(**grid_dict)
    set_xsection_axis_labels(ax)

    # colorbar for head
    cax = fx.ax.inset_axes(
        cbar_axis,
    )
    cbar = plt.colorbar(cb, orientation="horizontal", cax=cax)
    cbar.ax.tick_params(
        labelsize=5,
        labelcolor="black",
        color="white",
        length=5,
        pad=2,
    )
    cbar.ax.set_title(
        "Head (m)",
        pad=2.5,
        loc="left",
        fontdict=font_dict,
    )
    styles.add_annotation(
        ax=ax,
        text="A",
        xy=(0, 105),
        xytext=(15, 2),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="A'",
        xy=(ax.get_xlim()[1], 0),
        xytext=(-15, 17),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )

    # north-south cross-section
    ax = axs[3]
    styles.heading(ax=ax, idx=3)
    fx = flopy.plot.PlotCrossSection(
        model=gwf,
        ax=ax,
        line={"line": [(72500, extent[3]), (72500, extent[2])]},
    )
    cb = fx.plot_array(head, head=head, vmin=60, vmax=70)
    fx.plot_grid(**grid_dict)
    set_xsection_axis_labels(ax, skip_ylabel=True)

    # colorbar for head
    cax = fx.ax.inset_axes(
        cbar_axis,
    )
    cbar = plt.colorbar(cb, orientation="horizontal", cax=cax)
    cbar.ax.tick_params(
        labelsize=5,
        labelcolor="black",
        color="white",
        length=5,
        pad=2,
    )
    cbar.ax.set_title(
        "Head (m)",
        pad=2.5,
        loc="left",
        fontdict=font_dict,
    )
    styles.add_annotation(
        ax=ax,
        text="B",
        xy=(0, 80),
        xytext=(15, 15),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="B'",
        xy=(ax.get_xlim()[1], 68),
        xytext=(-11, 19),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )

    # legend
    ax = axs[4]
    xy0 = (-100, -100)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_axis_off()

    ax.axhline(xy0[0], color="blue", lw=0.5, label="River")
    ax.axhline(xy0[0], color="red", lw=1, ls=":", label="Cross-section line")
    ax.axhline(
        xy0[0], color=contour_color, lw=0.5, ls="--", label="Head contour (m)"
    )
    styles.graph_legend(
        ax,
        ncol=3,
        loc="lower center",
        labelspacing=0.5,
        columnspacing=0.6,
        handletextpad=0.3,
    )

    fig.align_labels()

    fpth = os.path.join("..", "doc", "figures", "grids_flopy_plots.png")
    plt.savefig(fpth, dpi=300);

### Export vtk

In [ ]:
model_output_dir = "temp_vtk/vtk"
gwf.export(model_output_dir, fmt="vtk", vertical_exageration=500.0, pvd=True)

In [ ]:
model_output_dir = "temp_vtk/vtk_smooth"
gwf.export(
    model_output_dir,
    fmt="vtk",
    smooth=True,
    vertical_exageration=500.0,
    pvd=True,
)